# Big Data Platform
## Assignment 3: ServerLess

**The goal of this assignment is to:**
- Understand and practice the details of Serverless

**Instructions:**
- Students will form teams of two people each, and submit a single homework for each team.
- The same score for the homework will be given to each member of your team.
- Your solution is in the form of a Jupyter notebook file (with extension ipynb).
- Images/Graphs/Tables should be submitted inside the notebook.
- The notebook should be runnable and properly documented. 
- Please answer all the questions and include all your code.
- You are expected to submit a clear and pythonic code.
- You can change functions signatures/definitions.

**Submission:**
- Submission of the homework will be done via Moodle by uploading (not Zip):
    - Jupyter Notebook
    - 2 Log files
    - Additional local scripts
- The homework needs to be entirely in English.
- The deadline for submission is on Moodle.
- Late submission won't be allowed.

  
- In case of identical code submissions - both groups will get a Zero. 
- Some groups might be selected randomly to present their code.

**Requirements:**  
- Python 3.6 should be used.  
- You should implement the algorithms by yourself using only basic Python libraries (such as numpy,pandas,etc.)

<br><br><br><br>

**Grading:**
- Q0 - 10 points - Setup
- Q1 - 40 points - Serverless MapReduceEngine
- Q2 - 20 points - MapReduce job to calculate inverted index
- Q3 - 30 points - Shuffle

`Total: 100`

<br><br>

In [36]:
# !pip install --quiet zipfile36
# !pip install names
# !pip install numpy
# !pip install scipy
# !pip install pandas
# !pip install lithops
!pip install ibm-cos-sdk

Looking in indexes: https://pypi.python.org/simple/


In [37]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

from lithops import FunctionExecutor
from lithops.storage.cloud_proxy import os as lithops_os
from lithops.storage.cloud_proxy import open as lithops_open

# general
import os
import time
import logging
import threading
from threading import Thread
import random
import warnings
import threading # you can use easier threading packages

# ml
import numpy as np
import scipy as sp
import pandas as pd

# visual
# import seaborn as sns
# import matplotlib.pyplot as plt

# notebook
from IPython.display import display

#random last names
import names

#SQL
import sqlite3
from sqlite3 import Error

In [17]:
random.seed(123)

# Question 0
## Setup

1. Navigate to IBM Cloud and open a trial account. No need to provide a credit card
2. Choose IBM Cloud Object Storage service from the catalog
3. Create a new bucket in IBM Cloud Object Storage
4. Create credentials for the bucket with HMAC (access key and secret key)
5. Choose IBM Cloud Functions service from the catalog and create a service


#### Lithops setup
1. By using “git” tool, install master branch of the Lithops project from
https://github.com/lithops-cloud/lithops
2. Follow Lithops documentation and configure Lithops against IBM Cloud Functions and IBM Cloud Object Storage
3. Configure Lithops log level to be in DEBUG mode
4. Run Hello World example by using Futures API and verify all is working properly.


#### IBM Cloud Object Storage setup
1. Upload all the input CSV files that you used in homework 2 into the bucket you created in IBM Cloud Object Storage


<br><br><br>

In [18]:
def hello(name, number):
    return f'hello {name} {number}'


def test():
    with FunctionExecutor() as fexec:
        fut = fexec.call_async(hello, ('World', 1))
        print(fut.result())

In [4]:
test()

2022-01-08 18:51:57,210 [INFO] lithops.config -- Lithops v2.5.8
2022-01-08 18:51:57,211 [DEBUG] lithops.config -- Loading configuration from /Users/rludan/git/BigDataHW3/.lithops_config
2022-01-08 18:51:57,214 [DEBUG] lithops.config -- Loading Serverless backend module: ibm_cf
2022-01-08 18:51:57,260 [DEBUG] lithops.config -- Loading Storage backend module: ibm_cos
2022-01-08 18:51:57,333 [DEBUG] lithops.storage.backends.ibm_cos.ibm_cos -- Creating IBM COS client
2022-01-08 18:51:57,334 [DEBUG] lithops.storage.backends.ibm_cos.ibm_cos -- Set IBM COS Endpoint to https://s3.eu-de.cloud-object-storage.appdomain.cloud
2022-01-08 18:51:57,335 [DEBUG] lithops.storage.backends.ibm_cos.ibm_cos -- Using access_key and secret_key
2022-01-08 18:51:57,658 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS client created - Region: eu-de
2022-01-08 18:51:57,659 [DEBUG] lithops.serverless.backends.ibm_cf.ibm_cf -- Creating IBM Cloud Functions client
2022-01-08 18:51:57,660 [DEBUG] lithops.ser

hello World 1


2022-01-08 18:52:02,191 [DEBUG] lithops.invokers -- ExecutorID 842866-0 - Async invoker 1 finished
2022-01-08 18:52:02,191 [DEBUG] lithops.invokers -- ExecutorID 842866-0 - Async invoker 0 finished


In [19]:
DB_FILE_NAME='mydb.db'
TEMP_FOLDER='./mapreducetemp'
FINAL_FOLDER='./mapreducefinal'
NUM_OF_RECORDS = 10
TEMP_RESULTS_TBL='temp_results'

In [ ]:
def upload_csv_to_bucket(path, key_name):
    try:
        cos.upload_file(Filename=path, Bucket="cloud-object-storage-8k-cos-standard-7nq", Key='input/' + str(key_name))
        print(f"Uploaded: {path}")
    except Exception as e:
        print("Unable to create text file: {0}".format(e))


def seeder(number):
    firstname = ['John', 'Dana', 'Scott', 'Marc', 'Steven', 'Michael', 'Albert', 'Johanna']
    city = ['NewYork', 'Haifa', 'Munchen', 'London', 'PaloAlto',  'TelAviv', 'Kiev', 'Hamburg']
    secondname = []
    for i in range(10):
        rand_name = names.get_last_name()
        secondname.append(rand_name)
    df = pd.DataFrame()
    df["firstname"] = np.random.choice(firstname, NUM_OF_RECORDS)
    df["secondname"] = np.random.choice(secondname, NUM_OF_RECORDS)
    df["city"] = np.random.choice(city, NUM_OF_RECORDS)

    df.to_csv('./MyCsv%s.csv' % number, index=False)

    abspath = os.path.abspath('./MyCsv%s.csv' % number)
    key_name = os.path.basename(abspath)
    upload_csv_to_bucket(abspath, key_name)


def create_cos_client():
    global cos
    # Constants for IBM COS values
    COS_ENDPOINT = "https://s3.eu-de.cloud-object-storage.appdomain.cloud"
    COS_API_KEY_ID = ""
    COS_INSTANCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/a306dcbad13145068a4e3898eb5928f5:a155c1bc-bddb-4006-9826-1f72f5fe243b:bucket:cloud-object-storage-8k-cos-standard-7nq"
    # Create client
    cos = ibm_boto3.client("s3",
                           ibm_api_key_id=COS_API_KEY_ID,
                           ibm_service_instance_id=COS_INSTANCE_CRN,
                           endpoint_url=COS_ENDPOINT,
                           config=Config(signature_version="oauth")
                           )

create_cos_client()

for i in range (20):
    seeder(i)

# Question 1
## Serverless MapReduceEngine

Modify MapReduceEngine from homework 2 into the MapReduceServerlessEngine where map and reduce tasks executed as a serverless actions, instead of local threads. In particular:
1. Deploy all map tasks as a serverless actions by using Lithops against IBM Cloud Functions.
2. Collect results from all map tasks and store them in the same SQLite as you used in MapReduceEngine and use the same code for the sort and shuffle phase.
3. Deploy reduce tasks by using Lithops against IBM Cloud Functions. Instead of persisting results from reduce tasks, return results back to the MapReduceServerlessEngine and proceed with the same workflow as in MapReduceEngine
4. Return results of reduce tasks to the user

**Please attach:**  
Text file with all log messages Lithops printed to console during the execution. Make
sure log level is set to DEBUG mode.

#### Code:

In [ ]:
try:
    os.mkdir(TEMP_FOLDER)
    os.mkdir(FINAL_FOLDER)
except Exception as e:
    print(f"folder(s) already exist(s): {e}")

In [264]:
sql_create_temp_results_table = """CREATE TABLE IF NOT EXISTS temp_results (
                                    key text,
                                    value text
                                    ); """

In [265]:
sql_group_by_key = """SELECT key, GROUP_CONCAT(value)
                      FROM temp_results GROUP BY key ORDER BY (key);"""

In [266]:
sql_drop_all_tables = """DROP TABLE temp_results;"""

In [267]:
def drop_temp_tables(conn):
    try:
        c = conn.cursor()
        c.execute(sql_drop_all_tables)
    except Error as e:
        print(e)

In [268]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

In [269]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [270]:
def get_grouped_values(conn):
    cur = conn.cursor()
    cur.execute(sql_group_by_key)

    rows = cur.fetchall()
        
    return rows

In [271]:
connection = create_connection(DB_FILE_NAME)
create_table(connection, sql_create_temp_results_table)

2.6.0


In [272]:
if connection is not None:
    # create temp_results table
    create_table(connection, sql_create_temp_results_table)
else:
    print("Error! cannot create the database connection.")

In [273]:
class ThreadExecution(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None
        
    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return

In [274]:
class MapReduceServerlessEngine():
    def execute(self, input_data, map_function, reduce_function, params):
        map_threads_list = []
        map_thread_num = 1
        
        for key in input_data:
            map_thread_exec = ThreadExecution(target=map_function, args=(key, params['column']), name=map_thread_num)
            map_thread_exec.start()
            map_threads_list.append(map_thread_exec)
            map_thread_num += 1
        
        for running_map_thread_exec in map_threads_list:
            map_result = running_map_thread_exec.join()
            
            if map_result is not None:
                map_result_df = pd.DataFrame(map_result, columns=["key", "value"])
                map_result_df.to_csv(TEMP_FOLDER + '/part-tmp-%s.csv' % running_map_thread_exec.name, index=False, header=True)
        
        for temp_file_name in os.scandir(TEMP_FOLDER):
            csv_df = pd.read_csv(temp_file_name.path)
            csv_df.to_sql(TEMP_RESULTS_TBL, connection, if_exists='append', index=False)
        
        grouped_values = get_grouped_values(connection)

        reduce_threads_list = []
        reduce_thread_num = 1
        
        for value in grouped_values:
            reduce_thread_exec = ThreadExecution(target=reduce_function, args=(value[0], value[1]), name=reduce_thread_num)
            reduce_thread_exec.start()
            reduce_threads_list.append(reduce_thread_exec)
            reduce_thread_num += 1
            
        for running_reduce_thread_exec in reduce_threads_list:
            reduce_result = running_reduce_thread_exec.join()
            
            if reduce_result is not None:
                result_df = pd.DataFrame(reduce_result, columns=["values"])
                result_df.to_csv(FINAL_FOLDER + '/part-%s-final.csv' % running_reduce_thread_exec.name, index=False, header=True)
            
        return "MapReduce Completed"

In [ ]:
def inverted_map(document_name, column_index):
    ret_val = []
    values = pd.read_csv(filepath_or_buffer = document_name, usecols=[column_index], skiprows=1)

    return [(x[0], document_name) for x in values.to_records(index=False)]

In [ ]:
def inverted_reduce(value, documents):
    ret_val = [value]
    temp_set = set(documents.split(','))
    ret_val.extend(temp_set)

    return ret_val

In [ ]:
input_data = []
for number in range(0,20):
    input_data.append('./MyCSV%s.csv' % number)

print(input_data)

In [ ]:
mapreduce = MapReduceEngine()
status = mapreduce.execute(input_data, inverted_map, inverted_reduce, params={'column':0})
print(status)

In [ ]:
# for file_name in os.scandir(TEMP_FOLDER):
#     os.remove(file_name.path)
# drop_temp_tables(connection)

# Task 2
## Submit MapReduce job to calculate inverted index
1. Use input_data: `cos://bucket/<path to CSV data>`
2. Submit MapReduce job with reduce and map functions as you used in homework 2, as follows

    `mapreduce = MapReduceServerlessEngine()`  
    `results = mapreduce.execute(input_data, inverted_map, inverted_index)`   
    `print(results)`

**Please attach:**  
Text file with all log messages Lithops printed to console during the execution. Make
sure log level is set to DEBUG mode.

#### Code:

# Question 3
## Shuffle

MapReduceServerlessEngine deploys both map and reduce tasks as serverless invocations.   
However, once map stage completed, the result are transferred from the map tasks to the SQLite database located on the client machine (laptop in your case), then performed local shuffle and then invoked reduce tasks passing them relevant parameters.

(To support your answers, feel free to use examples, Images, etc.)
<br><br>

**1. Explain why this approach is not efficient and what are cons and pros of such architecture in general. In broader scope you may assume that MapReduceServerlessEngine executed in some powerful machine and not just laptop.**

\<your answer here>

<br><br>
**2. Suggest how can you improve shuffle so intermediate data will not be downloaded to the client at all and shuffle performed in the cloud as well. Explain pros and cons of the approaches you suggest.**


\<your answer here>

<br><br>
**3. Can you make serverless shuffle?**


\<your answer here>

<br><br><br><br>
Good Luck :) 